In [1]:
from smartsim import Experiment
from smartsim.database import SlurmOrchestrator
from smartsim.settings import SrunSettings
from smartsim import slurm

def launch_cluster_orc(experiment, port, alloc):
    """Just spin up a database cluster, check the status
       and tear it down"""

    db = SlurmOrchestrator(port=port,
                            db_nodes=1,
                            batch=False,
                            alloc=alloc,
                            interface="ib0")
    

    # generate directories for output files
    # pass in objects to make dirs for
    experiment.generate(db, overwrite=True)

    # start the database on interactive allocation
    experiment.start(db, block=True)

    return db

def create_uploader(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="data_uploader.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    uploader = experiment.create_ensemble("uploader", replicas=2, run_settings=srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    uploader.attach_generator_files(to_copy=["./tf/data_uploader.py"])
    experiment.generate(uploader, overwrite=True)
    return uploader


def create_trainer(experiment, alloc):

    srun = SrunSettings(exe="python",
                        exe_args="training_service.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_tasks(1)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./tf/training_service.py")
    experiment.generate(trainer, overwrite=True)
    return trainer


def create_trainer_hvd(experiment, alloc, nodes=1, tasks_per_node=1):

    srun = SrunSettings(exe="python",
                        exe_args="training_service_hvd.py",
                        env_vars={"PYTHONUNBUFFERED": "1"},
                        alloc=alloc)
    srun.set_nodes(nodes)
    srun.set_tasks_per_node(tasks_per_node)

    trainer = experiment.create_model("trainer", srun)

    # create directories for the output files and copy
    # scripts to execution location inside newly created dir
    # only necessary if its not an executable (python is executable here)
    trainer.attach_generator_files(to_copy="./tf/training_service_hvd.py")
    experiment.generate(trainer, overwrite=True)
    return trainer

In [2]:
alloc = slurm.get_allocation(nodes=4, time="03:00:00", options={"constraint": "V100", "partition": "spider"})

14:09:55 osprey.us.cray.com SmartSim[35792] INFO Allocation successful with Job ID: 528404


In [9]:
exp = Experiment("launch_streaming", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 2)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer(exp, alloc)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

13:56:34 osprey.us.cray.com SmartSim[80887] INFO Working in previously created experiment
13:57:07 osprey.us.cray.com SmartSim[80887] INFO Working in previously created experiment
13:57:14 osprey.us.cray.com SmartSim[80887] INFO Working in previously created experiment
13:57:22 osprey.us.cray.com SmartSim[80887] INFO uploader_0(528403.13): New
13:57:22 osprey.us.cray.com SmartSim[80887] INFO uploader_1(528403.14): New
13:57:22 osprey.us.cray.com SmartSim[80887] INFO trainer(528403.15): New
13:57:27 osprey.us.cray.com SmartSim[80887] INFO uploader_0(528403.13): Running
13:57:27 osprey.us.cray.com SmartSim[80887] INFO uploader_1(528403.14): Running
13:57:27 osprey.us.cray.com SmartSim[80887] INFO trainer(528403.15): Running
13:57:32 osprey.us.cray.com SmartSim[80887] INFO uploader_0(528403.13): Running
13:57:32 osprey.us.cray.com SmartSim[80887] INFO uploader_1(528403.14): Running
13:57:32 osprey.us.cray.com SmartSim[80887] INFO trainer(528403.15): Running
13:57:37 osprey.us.cray.com Sma

KeyboardInterrupt: 

In [6]:
exp.stop(db, uploader_model, trainer_model)

14:18:13 osprey.us.cray.com SmartSim[35792] INFO Stopping model trainer with job name trainer-CG538LW7NOB7
14:18:13 osprey.us.cray.com SmartSim[35792] INFO Stopping model uploader_0 with job name uploader_0-CG538IVYD7OL
14:18:13 osprey.us.cray.com SmartSim[35792] INFO Stopping model uploader_1 with job name uploader_1-CG538IVYE6EU
14:18:13 osprey.us.cray.com SmartSim[35792] INFO Stopping model orchestrator_0 with job name orchestrator_0-CG5383JDMFVG


In [7]:
exp = Experiment("launch_streaming_hvd", launcher="slurm")

db_port = 6780

# start the database
db = launch_cluster_orc(exp, db_port, alloc)
uploader_model = create_uploader(exp, alloc, 1, 8)
uploader_model.enable_key_prefixing()
exp.start(uploader_model, block=False, summary=False)
trainer_model = create_trainer_hvd(exp, alloc, 1, 8)
for uploader in uploader_model.entities:
    trainer_model.register_incoming_entity(uploader)

exp.start(trainer_model, block=True, summary=False)

# shutdown the database because we don't need it anymore
exp.stop(db)

print(exp.summary())

14:18:14 osprey.us.cray.com SmartSim[35792] INFO Working in previously created experiment
14:18:47 osprey.us.cray.com SmartSim[35792] INFO Working in previously created experiment
14:18:53 osprey.us.cray.com SmartSim[35792] INFO Working in previously created experiment
14:19:02 osprey.us.cray.com SmartSim[35792] INFO uploader_0(528404.9): New
14:19:02 osprey.us.cray.com SmartSim[35792] INFO uploader_1(528404.10): New
14:19:02 osprey.us.cray.com SmartSim[35792] INFO trainer(528404.11): New
14:19:07 osprey.us.cray.com SmartSim[35792] INFO uploader_0(528404.9): Running
14:19:07 osprey.us.cray.com SmartSim[35792] INFO uploader_1(528404.10): Running
14:19:07 osprey.us.cray.com SmartSim[35792] INFO trainer(528404.11): Running
14:19:12 osprey.us.cray.com SmartSim[35792] INFO uploader_0(528404.9): Running
14:19:12 osprey.us.cray.com SmartSim[35792] INFO uploader_1(528404.10): Running
14:19:12 osprey.us.cray.com SmartSim[35792] INFO trainer(528404.11): Running
14:19:17 osprey.us.cray.com SmartS

In [16]:
slurm.release_allocation(alloc)

14:09:28 osprey.us.cray.com SmartSim[80887] INFO Releasing allocation: 528403
14:09:28 osprey.us.cray.com SmartSim[80887] INFO Successfully freed allocation 528403
